In [1]:
# !rm -rf data
![ ! -d 'data' ] && git clone https://github.com/ai-powered-search/tmdb.git data
!echo 'cloned'
! cd data && git pull
!echo 'pulled'

! cd data && tar -xvf movies.tgz  && tar -xvf judgments.tgz
![ ! -f 'data/title_judgments_logged.json.gz' ] && curl --location http://softwaredoug.com/data/title_judgments_logged.json.gz --output data/title_judgments_logged.json.gz

cloned
Already up to date.
pulled
x tmdb.json
x ai_pow_search_judgments.txt


In [121]:
import pandas as pd

tmdb = pd.read_json('data/tmdb.json').transpose()
tmdb['overview'] = tmdb['overview'].fillna('')
tmdb = tmdb.reset_index().rename(columns={'index': 'tmdb_id'})
tmdb.loc[tmdb['tmdb_id'] == 7555, :]

,tmdb_id,adult,backdrop_path,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,...,tagline,title,video,vote_average,vote_count,cast,directors,mlensId,status_message,status_code
16534,7555,False,/5W0ljXTtLVuLF0z8GE7mNXgTC44.jpg,"{'id': 5039, 'name': 'Rambo Collection', 'post...",10000000,"[{'id': 28, 'name': 'Action'}, {'id': 53, 'nam...",http://www.rambofilm.com/,7555,tt0462499,en,...,Heroes never die... They just reload.,Rambo,False,6.5,2212,"[{'cast_id': 12, 'character': 'John Rambo', 'c...","[{'credit_id': '52fe4483c3a36847f809a9af', 'de...",NaN,NaN,NaN


In [6]:
import pandas as pd


judgments = pd.read_json('data/title_judgments_logged.json.gz')
judgments = judgments.rename(columns={'docId': 'doc_id'})
judgments

,uid,qid,keywords,doc_id,grade,features
0,17555,1,rambo,7555,4,"[11.657399, 10.083591]"
1,11370,1,rambo,1370,3,"[9.456276, 13.265001]"
2,11369,1,rambo,1369,3,"[6.036743, 11.113943]"
3,113258,1,rambo,13258,2,"[0.0, 6.8695450000000005]"
4,11368,1,rambo,1368,4,"[0.0, 11.113943]"
...,...,...,...,...,...,...
1385,4037079,40,star wars,37079,0,"[0.0, 0.0]"
1386,40126757,40,star wars,126757,0,"[0.0, 0.0]"
1387,4039797,40,star wars,39797,0,"[0.0, 0.0]"
1388,4018112,40,star wars,18112,0,"[0.0, 0.0]"


In [7]:
judgments

,uid,qid,keywords,doc_id,grade,features
0,17555,1,rambo,7555,4,"[11.657399, 10.083591]"
1,11370,1,rambo,1370,3,"[9.456276, 13.265001]"
2,11369,1,rambo,1369,3,"[6.036743, 11.113943]"
3,113258,1,rambo,13258,2,"[0.0, 6.8695450000000005]"
4,11368,1,rambo,1368,4,"[0.0, 11.113943]"
...,...,...,...,...,...,...
1385,4037079,40,star wars,37079,0,"[0.0, 0.0]"
1386,40126757,40,star wars,126757,0,"[0.0, 0.0]"
1387,4039797,40,star wars,39797,0,"[0.0, 0.0]"
1388,4018112,40,star wars,18112,0,"[0.0, 0.0]"


In [8]:
import tensorflow_hub as hub


2023-01-16 09:46:06.154103: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [9]:
module_url = "https://tfhub.dev/google/universal-sentence-encoder/4" #@param ["https://tfhub.dev/google/universal-sentence-encoder/4", "https://tfhub.dev/google/universal-sentence-encoder-large/5"]
model = hub.load(module_url)

2023-01-16 09:46:10.648983: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-01-16 09:46:10.649480: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-01-16 09:46:10.649504: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Metal device set to: AMD Radeon Pro 5300M


2023-01-16 09:46:13.936911: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


In [10]:
model(["Mary had a little lamb"])

2023-01-16 09:46:15.702323: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


<tf.Tensor: shape=(1, 512), dtype=float32, numpy=
array([[-0.07028453,  0.01059935,  0.08423027, -0.0328695 ,  0.02756505,
         0.00913632,  0.0239979 , -0.02628415,  0.03472483,  0.0650179 ,
         0.01685206, -0.02107863, -0.00316442,  0.01285157, -0.02940562,
        -0.06912368, -0.04474806, -0.02484763,  0.03213454, -0.08085862,
        -0.00281348, -0.010038  , -0.07211809, -0.06478165, -0.06929238,
        -0.07281128,  0.01756493,  0.01702456,  0.02819746,  0.01535106,
         0.03689979, -0.07985415, -0.05297632, -0.00374063,  0.03171751,
        -0.07539806,  0.00078359,  0.00415806,  0.03809249, -0.00017633,
         0.07057564, -0.0275983 , -0.01971644, -0.03484123, -0.07448607,
         0.02777741, -0.02331628,  0.00193907, -0.07192903, -0.02623207,
         0.01451908, -0.05442308, -0.0376509 , -0.04132596,  0.0610454 ,
         0.04082571,  0.02590919, -0.08042567,  0.01143595,  0.00482652,
        -0.07489359, -0.06260989, -0.08479565,  0.04441264,  0.01578628,
 

## Compute our lambdas as predictors - what best optimizes DCG

In [11]:
lambdas_per_query = judgments.copy()


lambdas_per_query['last_prediction'] = 0.0  # 'relevance score' f(title, overview) | DCG 
lambdas_per_query.sort_values(['qid', 'last_prediction'], ascending=[True, False], kind='stable')

,uid,qid,keywords,doc_id,grade,features,last_prediction
0,17555,1,rambo,7555,4,"[11.657399, 10.083591]",0.0
1,11370,1,rambo,1370,3,"[9.456276, 13.265001]",0.0
2,11369,1,rambo,1369,3,"[6.036743, 11.113943]",0.0
3,113258,1,rambo,13258,2,"[0.0, 6.8695450000000005]",0.0
4,11368,1,rambo,1368,4,"[0.0, 11.113943]",0.0
...,...,...,...,...,...,...,...
1385,4037079,40,star wars,37079,0,"[0.0, 0.0]",0.0
1386,40126757,40,star wars,126757,0,"[0.0, 0.0]",0.0
1387,4039797,40,star wars,39797,0,"[0.0, 0.0]",0.0
1388,4018112,40,star wars,18112,0,"[0.0, 0.0]",0.0


In [12]:
import numpy as np

# Sort by our prediction
lambdas_per_query.sort_values(['qid', 'last_prediction'], ascending=[True, False], kind='stable')
lambdas_per_query['display_rank'] = lambdas_per_query.groupby('qid').cumcount()

# Compute stats for display rank
lambdas_per_query['discount'] = 1 / np.log2(2 + lambdas_per_query['display_rank']) # How much weight for this position
lambdas_per_query['gain'] = (2**lambdas_per_query['grade'] - 1)   # the 'gain' of this rank basically 'grade'

lambdas_per_query[['qid', 'display_rank', 'discount', 'grade', 'gain', 'features']]

,qid,display_rank,discount,grade,gain,features
0,1,0,1.000000,4,15,"[11.657399, 10.083591]"
1,1,1,0.630930,3,7,"[9.456276, 13.265001]"
2,1,2,0.500000,3,7,"[6.036743, 11.113943]"
3,1,3,0.430677,2,3,"[0.0, 6.8695450000000005]"
4,1,4,0.386853,4,15,"[0.0, 11.113943]"
...,...,...,...,...,...,...
1385,40,25,0.210310,0,0,"[0.0, 0.0]"
1386,40,26,0.208015,0,0,"[0.0, 0.0]"
1387,40,27,0.205847,0,0,"[0.0, 0.0]"
1388,40,28,0.203795,0,0,"[0.0, 0.0]"


In [13]:
import numpy as np

# each group paired with each other group
swaps = lambdas_per_query.merge(lambdas_per_query, on='qid', how='outer')
# Why is the diff not? This is what my slides say
# (discount x * gain x) - (discount y * gain y) + 
# delta_rank_1 + delta_rank_4
swaps['dcg_delta'] = np.abs((swaps['discount_x'] - swaps['discount_y']) * (swaps['gain_x'] - swaps['gain_y']))
swaps[['qid', 'discount_x', 'discount_y', 'gain_x', 'gain_y', 'display_rank_x', 'display_rank_y', 'dcg_delta']]

,qid,discount_x,discount_y,gain_x,gain_y,display_rank_x,display_rank_y,dcg_delta
0,1,1.000000,1.000000,15,15,0,0,0.000000
1,1,1.000000,0.630930,15,7,0,1,2.952562
2,1,1.000000,0.500000,15,7,0,2,4.000000
3,1,1.000000,0.430677,15,3,0,3,6.831881
4,1,1.000000,0.386853,15,15,0,4,0.000000
...,...,...,...,...,...,...,...,...
49019,40,0.201849,0.210310,0,0,29,25,0.000000
49020,40,0.201849,0.208015,0,0,29,26,0.000000
49021,40,0.201849,0.205847,0,0,29,27,0.000000
49022,40,0.201849,0.203795,0,0,29,28,0.000000


In [14]:
swaps['rho'] = 1 / (1 + np.exp(swaps['last_prediction_x'] - swaps['last_prediction_y']))
swaps[['qid', 'display_rank_x', 'display_rank_y', 'dcg_delta', 'last_prediction_x', 'last_prediction_y', 'rho']]

,qid,display_rank_x,display_rank_y,dcg_delta,last_prediction_x,last_prediction_y,rho
0,1,0,0,0.000000,0.0,0.0,0.5
1,1,0,1,2.952562,0.0,0.0,0.5
2,1,0,2,4.000000,0.0,0.0,0.5
3,1,0,3,6.831881,0.0,0.0,0.5
4,1,0,4,0.000000,0.0,0.0,0.5
...,...,...,...,...,...,...,...
49019,40,29,25,0.000000,0.0,0.0,0.5
49020,40,29,26,0.000000,0.0,0.0,0.5
49021,40,29,27,0.000000,0.0,0.0,0.5
49022,40,29,28,0.000000,0.0,0.0,0.5


In [15]:
swaps["lambda"] = 0

x_better = (swaps["grade_x"] > swaps["grade_y"])
swaps.loc[x_better, "lambda"] = swaps.loc[x_better, "dcg_delta"] * swaps.loc[x_better, "rho"]

swaps[["qid", "display_rank_x", "display_rank_y", "dcg_delta",
       "last_prediction_x", "last_prediction_y", "rho", "lambda",]]

,qid,display_rank_x,display_rank_y,dcg_delta,last_prediction_x,last_prediction_y,rho,lambda
0,1,0,0,0.000000,0.0,0.0,0.5,0.000000
1,1,0,1,2.952562,0.0,0.0,0.5,1.476281
2,1,0,2,4.000000,0.0,0.0,0.5,2.000000
3,1,0,3,6.831881,0.0,0.0,0.5,3.415941
4,1,0,4,0.000000,0.0,0.0,0.5,0.000000
...,...,...,...,...,...,...,...,...
49019,40,29,25,0.000000,0.0,0.0,0.5,0.000000
49020,40,29,26,0.000000,0.0,0.0,0.5,0.000000
49021,40,29,27,0.000000,0.0,0.0,0.5,0.000000
49022,40,29,28,0.000000,0.0,0.0,0.5,0.000000


In [16]:
# Better minus worse
lambdas_x = swaps.groupby(['qid', 'display_rank_x'])['lambda'].sum().rename('lambda')
lambdas_y = swaps.groupby(['qid', 'display_rank_y'])['lambda'].sum().rename('lambda')
lambdas = lambdas_x - lambdas_y
lambdas
lambdas_per_query = lambdas_per_query.merge(lambdas, left_on=['qid', 'display_rank'], right_on=['qid', 'display_rank_x'], how='left')
lambdas_per_query[['qid', 'doc_id', 'grade', 'features', 'lambda']]

,qid,doc_id,grade,features,lambda
0,1,7555,4,"[11.657399, 10.083591]",211.781688
1,1,1370,3,"[9.456276, 13.265001]",46.938369
2,1,1369,3,"[6.036743, 11.113943]",30.637615
3,1,13258,2,"[0.0, 6.8695450000000005]",5.888732
4,1,1368,4,"[0.0, 11.113943]",43.177578
...,...,...,...,...,...
1385,40,37079,0,"[0.0, 0.0]",-9.853045
1386,40,126757,0,"[0.0, 0.0]",-9.911575
1387,40,39797,0,"[0.0, 0.0]",-9.966853
1388,40,18112,0,"[0.0, 0.0]",-10.019174


## Create document embeddings

In [106]:
tmdb = tmdb.reset_index(drop=True)
tmdb

,adult,backdrop_path,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,...,tagline,title,video,vote_average,vote_count,cast,directors,mlensId,status_message,status_code
0,False,/zjLR7DbWujKiG7VhtjQ4r3gsjou.jpg,None,0,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",,43000,tt0055827,en,Le caporal épinglé,...,,The Elusive Corporal,False,5.9,10,"[{'cast_id': 3, 'character': 'Caporal', 'credi...","[{'credit_id': '52fe4626c3a36847f80ef68b', 'de...",NaN,NaN,NaN
1,False,None,None,0,"[{'id': 18, 'name': 'Drama'}, {'id': 37, 'name...",,43002,tt0056195,en,Lonely Are the Brave,...,Life can never cage a man like this!,Lonely Are the Brave,False,7.5,60,"[{'cast_id': 1, 'character': 'John W. ""Jack"" B...","[{'credit_id': '52fe4626c3a36847f80ef733', 'de...",NaN,NaN,NaN
2,False,/bQzj8DnIghVqUSe5EsH4uTV3jQb.jpg,None,500000,"[{'id': 18, 'name': 'Drama'}, {'id': 36, 'name...",,43004,tt0056196,en,Long Day's Journey Into Night,...,PRIDE...POWER...PASSION...PAIN!,Long Day's Journey Into Night,False,6.9,25,"[{'cast_id': 1, 'character': 'Mary Tyrone', 'c...","[{'credit_id': '52fe4626c3a36847f80ef791', 'de...",NaN,NaN,NaN
3,False,/gluooDaZ9PYOq4pUlKDCagRoBcp.jpg,None,0,"[{'id': 35, 'name': 'Comedy'}, {'id': 10749, '...",,43006,tt0056267,en,My Geisha,...,,My Geisha,False,6.1,7,"[{'cast_id': 2, 'character': 'Lucy Dell / Yoko...","[{'credit_id': '52fe4626c3a36847f80ef7ed', 'de...",NaN,NaN,NaN
4,False,/mKeTNV8NzpbfmovXcouEIcercYU.jpg,None,0,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",,43007,tt0056341,en,Period of Adjustment,...,The agonizing pause between the honeymoon and ...,Period of Adjustment,False,6.5,7,"[{'cast_id': 2, 'character': 'Ralph Bates', 'c...","[{'credit_id': '52fe4626c3a36847f80ef839', 'de...",NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65697,False,/lMbsU896qifmQxrGOvyqSPSFqDe.jpg,None,0,"[{'id': 18, 'name': 'Drama'}, {'id': 35, 'name...",,65485,tt0140603,it,La cena,...,,The Dinner,False,6.8,3,"[{'order': 0, 'id': 20234, 'cast_id': 2, 'cred...","[{'id': 31894, 'department': 'Directing', 'cre...",114685,NaN,NaN
65698,False,None,None,0,"[{'id': 18, 'name': 'Drama'}, {'id': 10769, 'n...",,76452,tt0069823,en,Una breve vacanza,...,,A Brief Vacation,False,5.0,3,"[{'order': 0, 'id': 83259, 'cast_id': 4, 'cred...","[{'id': 12329, 'department': 'Directing', 'cre...",69931,NaN,NaN
65699,False,/qfMXME2pjoRPY4mMl2CcGLE5DhM.jpg,None,0,"[{'id': 18, 'name': 'Drama'}, {'id': 14, 'name...",,65513,tt0378661,en,Les Revenants,...,Why have the dead suddenly returned...,They Came Back,False,5.6,16,"[{'order': 0, 'id': 19888, 'cast_id': 1, 'cred...","[{'id': 17387, 'department': 'Directing', 'cre...",36462,NaN,NaN
65700,False,/8wdIT4VHoEoqu5On94dzZan5t9a.jpg,None,0,"[{'id': 18, 'name': 'Drama'}, {'id': 35, 'name...",,65521,tt1926313,en,Pyaar Ka Punchnama,...,,Pyaar Ka Punchnama,False,7.5,14,"[{'order': 0, 'id': 1164344, 'cast_id': 4, 'cr...","[{'id': 1158937, 'department': 'Directing', 'c...",103050,NaN,NaN


In [122]:
import tensorflow as tf

def divide_chunks(l, n):
    for i in range(0, len(l), n):
        yield l[i:i + n]

# Seriously Tensorflow, I need to do this batching myself?
text = tf.convert_to_tensor(tmdb['overview'].to_list())
batch_size = 1000
embeddings = tf.zeros([0, 512], dtype=tf.float32)
for idx, batch in enumerate(divide_chunks(text, batch_size)):
    predictions = model.predict(batch)
    embeddings = tf.concat([embeddings, predictions['outputs']], axis=0)
    
overview_use = embeddings.numpy()

In [124]:
# Map doc_id -> row to find the right array in overview vector space
doc_ids = judgments['doc_id'].unique()
doc_ids

array([ 7555,  1370,  1369, ..., 39797, 18112, 43052])

In [125]:
tmdb[tmdb['id'].isin(doc_ids)]

,tmdb_id,adult,backdrop_path,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,...,tagline,title,video,vote_average,vote_count,cast,directors,mlensId,status_message,status_code
5,43008,False,/nbUnIkRlBmUEnI3Y9shgirevAvc.jpg,None,0,"[{'id': 37, 'name': 'Western'}]",,43008,tt0052876,en,...,From The Prize Novel -- A Picture Of Thunderin...,The Hanging Tree,False,6.5,42,"[{'cast_id': 2, 'character': 'Dr. Joseph 'Doc'...","[{'credit_id': '52fe4626c3a36847f80ef88b', 'de...",NaN,NaN,NaN
35,43052,False,None,None,0,"[{'id': 18, 'name': 'Drama'}]",,43052,tt0803080,en,...,,Säg att du älskar mig,False,4.6,4,[],"[{'credit_id': '54c2afa092514124ed004162', 'de...",NaN,NaN,NaN
103,43189,False,/wWO1oD6O1ytXKNQEBOsjRGyscvu.jpg,None,0,"[{'id': 18, 'name': 'Drama'}, {'id': 10749, 'n...",,43189,tt0044903,en,...,Inspired by the true story of bathing beauties!,Million Dollar Mermaid,False,6.0,8,"[{'cast_id': 1, 'character': 'Annette Kellerma...","[{'credit_id': '52fe4633c3a36847f80f2311', 'de...",NaN,NaN,NaN
164,43317,False,/umEJ78FUDu6DyOOYAgLjT48Hynk.jpg,None,1500000,"[{'id': 35, 'name': 'Comedy'}, {'id': 10402, '...",,43317,tt0047840,en,...,,Artists and Models,False,6.9,50,"[{'cast_id': 2, 'character': 'Rick Todd', 'cre...","[{'credit_id': '52fe463dc3a36847f80f3f43', 'de...",NaN,NaN,NaN
304,43522,False,/cuMRzYkcv8LgpGf2Oi0kHYY5HCX.jpg,None,0,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",,43522,tt0034778,en,...,"The grandest story of the Naughty ""Nineties"" b...",Gentleman Jim,False,7.0,25,"[{'cast_id': 2, 'character': 'James J. Corbett...","[{'credit_id': '52fe464ec3a36847f80f7cdd', 'de...",NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65469,82866,False,/tZvPiAZPHWvE6yjd8RI0b3CWWnm.jpg,None,0,"[{'id': 99, 'name': 'Documentary'}]",,82866,tt0996396,en,...,,Fog City Mavericks,False,0.0,0,"[{'order': 0, 'id': 1, 'cast_id': 1, 'credit_i...","[{'id': 223388, 'department': 'Directing', 'cr...",101133,NaN,NaN
65523,28323,False,/kcMcA9EbWfUtyHLMq6MdIHAWSaL.jpg,None,0,"[{'id': 28, 'name': 'Action'}, {'id': 18, 'nam...",,28323,tt0073138,en,...,,I Will Fight No More Forever,False,3.3,3,"[{'order': 0, 'id': 6577, 'cast_id': 1, 'credi...","[{'id': 35525, 'department': 'Directing', 'cre...",107638,NaN,NaN
65595,195617,False,None,None,0,[],,195617,tt0021004,en,...,,It's a Bird,False,8.0,2,"[{'order': 0, 'id': 1067965, 'cast_id': 2, 'cr...","[{'id': 1067966, 'department': 'Directing', 'c...",125956,NaN,NaN
65674,130745,False,None,None,0,"[{'id': 99, 'name': 'Documentary'}]",,130745,tt2234419,en,...,,16 Acres,False,6.5,2,[],"[{'id': 126162, 'department': 'Directing', 'cr...",110582,NaN,NaN


## Create query embeddings

Create an embedding for each query in the document space

In [149]:
def get_embedding(tmdb_id):
    return overview_use[tmdb[tmdb['tmdb_id'] == tmdb_id].index]


# Create weighted query embeddings
judgments['weight'] = judgments['grade'] / 4
query_embeddings = {}
for query in lambdas_per_query['keywords'].unique():
    query_judgments = judgments[lambdas_per_query['keywords'] == query]
    for query_judgment in query_judgments.to_dict(orient='records'):
        tmdb_id = query_judgment['doc_id']
        weight = query_judgment['weight']
        doc_embedding = get_embedding(tmdb_id)[0]

        try:
            query_embeddings[query] += (weight * doc_embedding)
        except KeyError:
            query_embeddings[query] = weight * doc_embedding 
    query_embeddings[query] /= len(query_judgments)  # Not sure this matters
    query_embeddings[query] /= np.linalg.norm(query_embeddings[query])

In [151]:
query_embeddings['rambo']

array([ 0.02972494, -0.0147243 , -0.01402866, -0.03362493,  0.01442135,
        0.04483698,  0.04726744,  0.00158264,  0.05375212, -0.0581939 ,
        0.10194699,  0.06162515, -0.04879327,  0.02698465,  0.07649798,
       -0.08917999, -0.05672573,  0.03303665, -0.02343745,  0.03620373,
       -0.05647273, -0.03655627,  0.04265799, -0.01830724,  0.03702894,
        0.05074221, -0.02568416, -0.06261772, -0.00850914,  0.04832797,
        0.04672624, -0.04242633, -0.00931135, -0.0608523 , -0.01512212,
        0.09809384,  0.03624721, -0.00673144, -0.0223287 ,  0.01622309,
       -0.00770243, -0.06569903, -0.03214641, -0.06165315, -0.04767733,
        0.01726217,  0.03942685,  0.0200977 ,  0.04979463,  0.07223891,
       -0.0745049 , -0.00835147, -0.00082126, -0.00058608, -0.05878871,
       -0.0275895 , -0.08060712,  0.06870215,  0.06853117,  0.03632756,
        0.09723197,  0.09361825, -0.09854262, -0.01641996,  0.04028917,
       -0.01171924, -0.01762672, -0.04625538, -0.00267575,  0.03

In [160]:
def get_top_n(dotted, n=10):
    n = min(n, dotted.shape[0])
    top_n = np.argpartition(-dotted, n-1)[:n]
    scores = dotted[top_n]
    return sorted(zip(top_n, scores),
                  key=lambda scored: scored[1],
                  reverse=True)

def nn(query_vector,
       matrix,
       n=10):
    """ nth nearest neighbors as array
        with indices of nearest neighbors"""
    dotted = np.dot(matrix, query_vector)
    # print(f">> Dot - {perf_counter() - start}")
    top_n = get_top_n(dotted, n=n)
    # print(f">> Tpn - {perf_counter() - start}")
    return top_n

nearest_neighbors_rambo = nn(query_embeddings['rambo'], overview_use, 10)

In [164]:
def keys(top):
    return [tp[0] for tp in top]

tmdb.iloc[keys(nearest_neighbors_rambo), :][['title', 'tmdb_id', 'overview']]

,title,tmdb_id,overview
16534,Rambo,7555,When governments fail to act on behalf of capt...
50102,First Blood,1368,When former Green Beret John Rambo is harassed...
50104,Rambo III,1370,"Combat has taken its toll on Rambo, but he's f..."
50103,Rambo: First Blood Part II,1369,John Rambo is released from prison by the gove...
26114,Missing in Action,15379,American servicemen are still being held capti...
12937,The Bunker,261492,"""The Bunker,"" is a fictional war torn valor st..."
19200,Thou Shalt Not Kill... Except,40042,"Vietnam, 1969. War is Hell. For Marine Sergea..."
35510,Some Kind of Hero,37607,A Vietnam vet returns home from a prisoner of ...
36563,Phantom Raiders,110997,Deep within the steaming tropical jungles of V...
50603,The Soldier,298073,The Soldier tells us about the military operat...


In [158]:
judgments[judgments['keywords'] == 'rambo']

,uid,qid,keywords,doc_id,grade,features,grade_scaled,weight
0,17555,1,rambo,7555,4,"[11.657399, 10.083591]",1.00,1.00
1,11370,1,rambo,1370,3,"[9.456276, 13.265001]",0.75,0.75
2,11369,1,rambo,1369,3,"[6.036743, 11.113943]",0.75,0.75
3,113258,1,rambo,13258,2,"[0.0, 6.8695450000000005]",0.50,0.50
4,11368,1,rambo,1368,4,"[0.0, 11.113943]",1.00,1.00
5,131362,1,rambo,31362,1,"[0.0, 7.8627386]",0.25,0.25
6,161410,1,rambo,61410,1,"[0.0, 4.563677]",0.25,0.25
7,1319074,1,rambo,319074,0,"[0.0, 0.0]",0.00,0.00
8,110296,1,rambo,10296,0,"[0.0, 0.0]",0.00,0.00
9,135868,1,rambo,35868,0,"[0.0, 0.0]",0.00,0.00
